In [18]:
class SOSGame:
    def __init__(self, n=8, mode="Simple"):
        self.n = n
        self.mode = mode
        self.board = [['' for _ in range(n)] for _ in range(n)]
        self.current_player = 'Blue'
        self.symbol_choice = {'Blue': 'S', 'Red': 'O'}
    
    def place_symbol(self, row, col):
        if self.board[row][col] == '':
            self.board[row][col] = (self.symbol_choice[self.current_player], self.current_player)
            self.switch_turn()
    
    def switch_turn(self):
        self.current_player = 'Red' if self.current_player == 'Blue' else 'Blue'
    
    def swap_roles(self):
        if self.symbol_choice['Blue'] == 'S':
            self.symbol_choice = {'Blue': 'O', 'Red': 'S'}
        else:
            self.symbol_choice = {'Blue': 'S', 'Red': 'O'}
        print(f"Roles swapped: Blue = {self.symbol_choice['Blue']}, Red = {self.symbol_choice['Red']}")
    
    def set_game_mode(self, mode):
        self.mode = mode
        print(f"Game mode set to: {mode}")
    
    def set_board_size(self, new_size):
        try:
            new_size = int(new_size)
            if new_size > 2:
                self.n = new_size
                self.board = [['' for _ in range(new_size)] for _ in range(new_size)]
                print(f"Board size updated to: {new_size}")
                return True
        except ValueError:
            pass
        return False

    def reset_game(self):
        self.board = [['' for _ in range(self.n)] for _ in range(self.n)]
        self.current_player = 'Blue'
        print("Game reset")